## 전체 구조 요약
1. 문서 로드 (JSON, TXT 등)
2. 청크 분할 (TextSplitter)
3. 임베딩 (Upstage Embedding 사용)
4. 벡터 저장소 (FAISS)
5. 질문 입력 → 유사 문서 검색
6. LLM에게 prompt 생성 → 답변 출력

## 필요 라이브러리

In [1]:
import os
import pandas as pd
import json
import requests
import faiss
import numpy as np
from tqdm import tqdm


## api_key 저장

In [2]:
api_key = "up_kntyD3xQnR27odlyiYxbsL1c3HsfP"

## 데이터 처리
### 정부 혜택 json 가져오기

In [4]:

# ✅ 1. 현재 작업 디렉토리 기준 경로 설정
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "../EDA/data")

# ✅ 2. 파일명 고정
COMBINE_FILENAME = "combined_service_data"

combine_json_path = os.path.join(DATA_DIR, f"{COMBINE_FILENAME}.json")

# ✅ 3. JSON 불러오기
combine_df = pd.read_json(combine_json_path)

print("✅ JSON 파일 로딩 완료")
print(f"🟢 combine_df: {combine_df.shape}")

✅ JSON 파일 로딩 완료
🟢 combine_df: (26613, 20)


### json을 문자열로 바꾸기

In [7]:
docs = combine_df.apply(lambda row: json.dumps(row.to_dict(), ensure_ascii=False), axis=1).tolist()


### docs 시각화

In [8]:
condition_only_df = pd.DataFrame([json.loads(doc)["조건"] for doc in docs])
condition_only_df.head()


,남성,여성,중위소득 0~50%,중위소득 51~75%,중위소득 76~100%,중위소득 101~200%,중위소득 200% 초과,예비부모/난임,임산부,출산/입양,...,"농업,임업 및 어업",기타업종,기관/단체,제조업,영업중,사회복지시설,정보통신업,음식적업,예비창업자,생계곤란/폐업예정자
0,True,True,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,True,True,True,True,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,True,True,True,True,True,NaN,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,True,True,True,True,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,True,True,True,True,True,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Solar 임베딩
#### 텍스트를 벡터로 변환하는 함수
문서 검색용 벡터화를 담당하는 핵심 함수

In [11]:
def get_embedding(text, api_key):
    url = "https://api.upstage.ai/v1/embeddings"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "embedding-passage",  # 문서용 모델
        "input": text
    }

    response = requests.post(url, headers=headers, json=payload)

    try:
        return response.json()["data"][0]["embedding"]
    except Exception as e:
        print("❌ 파싱 실패:", e)
        print("📨 응답:", response.text[:300])
        return None


#### 임베딩 비용 추정

In [15]:
# 토큰 수 예측 (한글 기준 1토큰 ≈ 4자)
estimated_tokens = [len(doc) / 4 for doc in docs]
total_tokens = int(np.sum(estimated_tokens))

# Solar 실제 단가 적용
per_token_price = 0.09 / 969_007  # 실제 Solar 기준
estimated_cost = total_tokens * per_token_price

# 출력
print(f"📄 총 문서 수: {len(docs):,}")
print(f"🔢 총 예상 토큰 수: {total_tokens:,} tokens")
print(f"💸 예측 비용: ${estimated_cost:.4f} USD")

📄 총 문서 수: 26,613
🔢 총 예상 토큰 수: 7,586,500 tokens
💸 예측 비용: $0.7046 USD


In [16]:
embeddings = []
valid_indices = []

for i, doc in enumerate(tqdm(docs, desc="🔄 Embedding 중")):
    vec = get_embedding(doc, api_key)
    if vec:
        embeddings.append(vec)
        valid_indices.append(i)

print(f"✅ 총 {len(embeddings)}개 임베딩 생성 완료")

🔄 Embedding 중:   0%|          | 71/26613 [01:00<6:18:33,  1.17it/s]


KeyboardInterrupt: 